In this file we add the "committees" column to EP9 and EP10 voted main docs by looking at the API of How They Vote

In [9]:
import csv
import math
import time
import requests
from pathlib import Path

OUT_CSV = Path("data/all_votes_with_subjects.csv")
OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
BASE_API = "https://howtheyvote.eu/api/votes"
HEADERS = {"User-Agent": "EP-research/1.0"}
PAGE_SIZE = 200

session = requests.Session()

def flatten_subjects(vote):
    """Extract OEIL subject labels as a single string."""
    subjects = []
    for subj in vote.get("oeil_subjects", []):
        label = subj.get("label")
        if label:
            subjects.append(label.strip())
    if not subjects:
        # fallback to eurovoc or geo_areas if relevant
        eurovoc = [e.get("label") for e in vote.get("eurovoc_concepts", []) if e.get("label")]
        if eurovoc:
            subjects.extend(eurovoc)
    return "; ".join(subjects) if subjects else ""

def flatten_committees(vote):
    committees = vote.get("responsible_committees", [])
    labels = []
    codes = []
    for c in committees:
        if c.get("label"):
            labels.append(c["label"].strip())
        if c.get("code"):
            codes.append(c["code"].strip())
    return labels, codes

def flatten_geo(vote):
    """Return comma-separated geographic labels (countries/regions)."""
    geos = [g.get("label") for g in vote.get("geo_areas", []) if g.get("label")]
    return ", ".join(geos)

def normalize_vote(v):
    vid = v.get("id")
    title = v.get("display_title") or v.get("title") or ""
    timestamp = v.get("timestamp")
    subjects = flatten_subjects(v)
    labels, codes = flatten_committees(v)
    geo = flatten_geo(v)
    csv_url = f"https://howtheyvote.eu/api/votes/{vid}.csv"
    return {
        "id": vid,
        "title": title.strip(),
        "timestamp": timestamp,
        "subjects": subjects,
        "geo_areas": geo,
        "committee_labels": labels,   # <-- list, not string
        "committee_codes": codes,     # <-- list, not string
        "csv_url": csv_url,
    }

def fetch_page(page: int):
    params = {"page": page, "page_size": PAGE_SIZE, "sort_by": "timestamp", "sort_order": "desc"}
    r = session.get(BASE_API, params=params, headers=HEADERS, timeout=60)
    r.raise_for_status()
    return r.json()

def main():
    first = fetch_page(1)
    total = first.get("total") or first.get("count") or len(first.get("results", []))
    total_pages = math.ceil(total / PAGE_SIZE)
    print(f"Found {total} votes across ~{total_pages} pages")

    fieldnames = ["id", "title", "timestamp", "subjects", "geo_areas", "csv_url", "committee_labels", "committee_codes"]

    with OUT_CSV.open("w", newline="", encoding="utf-8") as fcsv:
        writer = csv.DictWriter(fcsv, fieldnames=fieldnames)
        writer.writeheader()

        for page in range(1, total_pages + 1):
            data = fetch_page(page)
            results = data.get("results", [])
            if not results:
                break

            for v in results:
                row = normalize_vote(v)
                row["committee_labels"] = ";".join(row["committee_labels"])
                row["committee_codes"] = ";".join(row["committee_codes"])
                writer.writerow(row)

            print(f"Page {page}/{total_pages} done ({len(results)} votes)")
            time.sleep(0.2)

    print(f"✅ Done.\nCSV saved to {OUT_CSV}\n")

if __name__ == "__main__":
    main()

Found 2096 votes across ~11 pages
Page 1/11 done (200 votes)
Page 2/11 done (200 votes)
Page 3/11 done (200 votes)
Page 4/11 done (200 votes)
Page 5/11 done (200 votes)
Page 6/11 done (200 votes)
Page 7/11 done (200 votes)
Page 8/11 done (200 votes)
Page 9/11 done (200 votes)
Page 10/11 done (200 votes)
Page 11/11 done (96 votes)
✅ Done.
CSV saved to data/all_votes_with_subjects.csv



In [12]:
# list unique committees from the csv
import pandas as pd

# Read the CSV file
df = pd.read_csv(OUT_CSV)

# list unique committees
committees = df['committee_labels'].unique()
print(len(committees))
print(committees)


58
['International Trade' 'Budgets' 'Economic and Monetary Affairs'
 'Legal Affairs' 'Foreign Affairs' 'Environment, Climate and Food Safety'
 nan 'Women’s Rights and Gender Equality' 'Constitutional Affairs'
 'Budgets;Economic and Monetary Affairs' 'Industry, Research and Energy'
 'Budgetary Control'
 'Agriculture and Rural Development;Environment, Climate and Food Safety'
 'Internal Market and Consumer Protection'
 'Civil Liberties, Justice and Home Affairs'
 'Employment and Social Affairs' 'Agriculture and Rural Development'
 'Fisheries' 'Transport and Tourism' 'Culture and Education'
 'Regional Development'
 'Environment, Climate and Food Safety;Internal Market and Consumer Protection'
 'Development;Environment, Climate and Food Safety'
 'Economic and Monetary Affairs;Civil Liberties, Justice and Home Affairs'
 'International Trade;Industry, Research and Energy' 'Petitions'
 'Foreign Affairs;Budgets'
 'Employment and Social Affairs;Regional Development'
 'European Parliament delega

In [14]:
import pandas as pd

# Load your CSV
df = pd.read_csv("data/all_votes_with_subjects.csv")

# Extract first committee into "policy_area"
df["policy_area"] = (
    df["committee_labels"]
    .fillna("")
    .astype(str)
    .str.split(";")
    .str[0]
    .str.strip()
    .replace("", pd.NA)
)

# Save back to CSV
df.to_csv("data/all_votes_with_subjects.csv", index=False)

print("Done. 'policy_area' column written to CSV.")

Done. 'policy_area' column written to CSV.


In [ ]:
import pandas as pd

df_policy = pd.read_csv("data/all_votes_with_subjects.csv")

# Ensure proper type & keep only id + policy_area
df_policy = df_policy[["id", "policy_area"]].rename(columns={"id": "id"})
df_policy["id"] = df_policy["id"].astype(str)

df_ep9 = pd.read_csv("data/votewatch_csv/EP9_Voted main docs.csv")
df_ep10 = pd.read_csv("data/votewatch_csv/EP10_Voted main docs.csv")

df_ep9["id"] = df_ep9["id"].astype(str)
df_ep10["id"] = df_ep10["id"].astype(str)

df_ep9 = df_ep9.merge(df_policy, on="id", how="left")
df_ep10 = df_ep10.merge(df_policy, on="id", how="left")

df_ep9.to_csv("data/votewatch_csv/EP9_Voted main docs.csv", index=False)
df_ep10.to_csv("data/votewatch_csv/EP10_Voted main docs.csv", index=False)


    

In [39]:
# list unique committees from the csv
import pandas as pd

# Read the CSV file
filename = "data/votewatch_csv/EP9_Voted main docs.csv"
df = pd.read_csv(filename)

# list unique committees
committees = df['policy_area'].unique()
print(len(committees))
print(committees)

# Print number of times each policy area is mentioned
policy_area_counts = df['policy_area'].value_counts()
for policy_area, count in policy_area_counts.items():
    print(f"Policy area '{policy_area}': mentioned {count} times")

# items with no policy area
print(df[df['policy_area'].isna()])

24
['Foreign Affairs' 'Budgets' 'Economic and Monetary Affairs'
 'Women’s Rights and Gender Equality' 'International Trade'
 'European Parliament delegation to the Budgetary Conciliation Committee'
 'Employment and Social Affairs' 'Agriculture and Rural Development'
 'Environment, Climate and Food Safety' 'Budgetary Control'
 'Civil Liberties, Justice and Home Affairs' 'Petitions'
 'Constitutional Affairs' 'Industry, Research and Energy'
 'Transport and Tourism' 'Legal Affairs' 'Development'
 'Regional Development' 'Internal Market and Consumer Protection'
 'Culture and Education' 'Fisheries' 'Special Committee on Beating Cancer'
 'Special Committee on Foreign Interference in all Democratic Processes in the European Union, including Disinformation'
 'Special Committee on Artificial Intelligence in a Digital Age']
Policy area 'Budgetary Control': mentioned 334 times
Policy area 'Foreign Affairs': mentioned 311 times
Policy area 'Environment, Climate and Food Safety': mentioned 188 times

In [37]:
import pandas as pd

# Load files
df_src = pd.read_csv("data/votewatch_csv/EP10_Voted_main_docs_with_committees_updated.csv")
df_dst = pd.read_csv("data/votewatch_csv/EP10_Voted main docs.csv")

# Ensure id/vote_number are strings
df_src["id"] = df_src["id"].astype(str)
df_dst["id"] = df_dst["id"].astype(str)

# Keep only id + committees (policy_area)
df_src2 = df_src[["id", "committees"]].rename(columns={"committees": "policy_area"})
print(df_src2)

# Merge to fill missing values
df_merged = df_dst.merge(df_src2, on="id", how="left", suffixes=("", "_from_src"))

# Fill policy_area only where missing in destination
df_merged["policy_area"] = df_merged["policy_area"].fillna(df_merged["policy_area_from_src"])

# Drop helper column
df_merged = df_merged.drop(columns=["policy_area_from_src"])

# Save final result
df_merged.to_csv("data/votewatch_csv/EP10_Voted main docs_filled.csv", index=False)

print("Done. Missing policy_area values have been filled.")

        id                policy_area
0   169362  foreign & security policy
1   169541  foreign & security policy
2   169544  foreign & security policy
3   169565            gender equality
4   169623  foreign & security policy
..     ...                        ...
75  179775  foreign & security policy
76  179776  foreign & security policy
77  179918  foreign & security policy
78  180245  foreign & security policy
79  180598  foreign & security policy

[80 rows x 2 columns]
Done. Missing policy_area values have been filled.
